In [1227]:
import math
import torch
import numpy as np
from PIL import Image
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import sklearn
import sklearn.datasets
from torch.functional import split
from sklearn.model_selection import train_test_split
import pandas as pd

In [1228]:
#!pip install sklearn.datasets

In [1229]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1230]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)

In [1231]:
class MyOwnFCH(torch.utils.data.Dataset):
   
    def __init__(self, features, target, transform=None):
        self.features = features
        self.target = target
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        features = self.features[idx]
        target = self.target[idx]
        return features, target

In [1232]:
df_features = pd.DataFrame(california_housing.data)
df_target = pd.DataFrame(california_housing.target)

In [1233]:
# california_housing.target_names
# california_housing.feature_names

In [1234]:
# for f in california_housing.feature_names:
#   print(len(df_features[df_features[f].isna() == True]))
# print(df_target[df_target['MedHouseVal'].isna() == True].count())

In [1235]:
# df_target

In [1236]:
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.25, random_state=13)

In [1237]:
from torch.utils.data import DataLoader

In [1238]:
# X_train.values.astype(np.float32)

In [1239]:
mo_fch_train = MyOwnFCH(X_train.values.astype(np.float32), y_train.values.astype(np.float32))
mo_fch_test = MyOwnFCH(X_test.values.astype(np.float32), y_test.values.astype(np.float32))

In [1240]:
batch_size = 128

mo_fch_train_loader = DataLoader(mo_fch_train, batch_size=batch_size,shuffle=False)
mo_fch_test_loader = DataLoader(mo_fch_test, batch_size=batch_size,shuffle=False)

In [1241]:
mo_fch_train_loader

In [1242]:
for x, l in mo_fch_train_loader:
    print(x.shape)
    print(l.shape)
    print(l[0])
    break

torch.Size([128, 8])
torch.Size([128, 1])
tensor([2.6800])


In [1243]:
for x, l in mo_fch_test_loader:
    print(x.shape)
    print(l.shape)
    print(l[0])
    break

torch.Size([128, 8])
torch.Size([128, 1])
tensor([1.3010])


In [1244]:
# classes = ['0','1']

In [1245]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=10):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 16 * hidden_dim)
        # self.bn1 = nn.BatchNorm1d(16 * hidden_dim)
        self.fc2 = nn.Linear(16 * hidden_dim, 8 * hidden_dim)
        # self.bn2 = nn.BatchNorm1d(8 * hidden_dim)
        self.fc3 = nn.Linear(8 * hidden_dim, 4 * hidden_dim)
        # self.bn3 = nn.BatchNorm1d(4 * hidden_dim)
        self.fc4 = nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        # self.bn4 = nn.BatchNorm1d(2 * hidden_dim)
        self.fc5 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.bn5 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc6 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        # x = self.bn1(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        # x = self.bn2(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        # x = self.bn3(x)
        x = self.fc4(x)
        x = F.leaky_relu(x)
        # x = self.bn4(x)
        x = self.fc5(x)
        x = F.leaky_relu(x)
        x = self.bn5(x)
        # x = F.leaky_relu(x)
        x = self.dp(x)
        # x = F.leaky_relu(x)
        x = self.fc6(x)
        return x
    
    def predict(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        # x = self.bn1(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        # x = self.bn2(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        # x = self.bn3(x)
        x = self.fc4(x)
        # x = F.softmax(x)
        x = F.leaky_relu(x)
        # x = self.bn4(x)
        x = self.fc5(x)
        x = F.leaky_relu(x)
        x = self.bn5(x)
        x = self.dp(x)
        # x = F.leaky_relu(x)
        x = self.fc6(x)
        return x

In [1246]:
net = Net(8,8)

In [1247]:
# optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
# optimizer = torch.optim.RMSprop(net.parameters(), lr=0.01)
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.PoissonNLLLoss()

In [1248]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'

In [1249]:
epochs = 5
model = net.to(device)
model.train()
# th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(mo_fch_train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(mo_fch_train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
            for j, data in enumerate(mo_fch_test_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте
                test_running_total += len(data[1])
                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_running_right += (test_labels == pred_test_labels).sum()
            
            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/121]. Loss: 4.387. Acc: 0.000. Test loss: 2290.250. Test acc: 0.014
Epoch [2/5]. Step [1/121]. Loss: 0.558. Acc: 0.000. Test loss: 0.668. Test acc: 0.037
Epoch [3/5]. Step [1/121]. Loss: 0.548. Acc: 0.000. Test loss: 0.677. Test acc: 0.037
Epoch [4/5]. Step [1/121]. Loss: 0.537. Acc: 0.000. Test loss: 0.682. Test acc: 0.037
Epoch [5/5]. Step [1/121]. Loss: 0.536. Acc: 0.000. Test loss: 0.678. Test acc: 0.037
Training is finished!


# Вывод

Результат очень сильно зависел от параметра batch_size, почти все значения loss.item уходили в NaN. Лучше всех с подсчётом точности справилась функция PoissonNLLLoss(), из тех, что походили по смыслу. 

Честно говоря, не поняла, почему у меня настолько не получилось обработать данные :( Вроде, никаких выбросов или NaN значений в самих данных не было, как и ненормально больших значений (прочитала, что это один из вариантов, почему loss functions могут принимать неопределённые или бесконечные значения). 
  
  Относительно лучшим оптимизатором оказался Adam. Без BatchNorm и Dropout слоёв результаты заметно ухудшались и потери всегда считались NaNами